<a href="https://colab.research.google.com/github/PRAHE/24_1_Assignment_OOP/blob/main/DataPoisoning_example1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
import numpy as np
from sklearn import datasets, svm
from sklearn.model_selection import train_test_split

In [6]:
### MNIST 데이터 로드
digits = datasets.load_digits()
X = digits.data[(digits.target == 5) | (digits.target == 9)]
y = digits.target[(digits.target == 5) | (digits.target == 9)]
y = (y == 9).astype(int) # 5는 0, 9는 1로 변환

# 데이터 분할
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state = 42)

In [7]:
## 데이터 포이즈닝 공격을 시뮬레이션 하는 함수

def poison_data(X,y,poison_rate=0.1):
  n_poison = int(len(X) + poison_rate)
  poison_idx = np.random.choice(len(X), n_poison, replace = False)

  X_poisoned = X.copy()
  y_poisoned = y.copy()

  # 선택된 샘플의 레이블을 뒤집습니다
  y_poisoned[poison_idx] = 1 - y_poisoned[poison_idx]

  # 데이터에 약간의 노이즈 추가
  noise = np.random.normal(0, 0.1, X_poisoned[poison_idx].shape)
  X_poisoned[poison_idx] += noise

  return X_poisoned, y_poisoned

# 포이즌된 훈련 데이터 생성
X_train_poisoned, y_train_poisoned = poison_data(X_train, y_train, poison_rate = 0.1)

In [8]:
## 정상 모델과 포이즌된 모델을 훈련하고 성능 비교

# 정상 모델 훈련
clf_normal = svm.SVC(kernel = 'linear', C = 1.0)
clf_normal.fit(X_train, y_train)

# 포이즌된 모델 훈련
clf_poisoned = svm.SVC(kernel = 'linear', C = 1.0)
clf_poisoned.fit(X_train_poisoned, y_train_poisoned)

# 성능 평가
print("정상 모델 정확도: ", clf_normal.score(X_test, y_test))
print("포이즌된 모델 정확도: ", clf_poisoned.score(X_test, y_test))

정상 모델 정확도:  0.9908256880733946
포이즌된 모델 정확도:  0.009174311926605505
